In [1]:
from sklearn import datasets, linear_model, metrics 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from mord import OrdinalRidge
from mord import LAD

Using TensorFlow backend.
C:\Users\65842\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\65842\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\65842\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\65842\Anaconda3\lib\site-packages\tensorboard\comp

In [2]:
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.layers import SpatialDropout1D
from keras.layers import Embedding
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [ ]:
data = pd.read_csv("librosa_mfcc_features_2.csv", index_col='Participant_ID')

In [ ]:
data.head()

In [ ]:
def remove_brackets(s):
    new_s = s[1:-1]
    return new_s

def str_to_np(X):
    new_X = []
    for i in range(len(X)):
        x_i = np.fromstring(X.iloc[i], dtype=float, sep='\t')
        new_X.append(x_i)
    return np.stack( new_X, axis=0 )

In [ ]:
data['feature'] = data['feature'].apply(remove_brackets)
X = data['feature']

In [ ]:
X.head()

In [ ]:
X = str_to_np(X)

In [ ]:
y = data['PHQ8_Binary'].values

In [ ]:
def balance_data(X,y):
    new_y = []
    new_X = []
    dep_count = sum(y)
    count = 0
    for i in range(X.shape[0]):
        if y[i]==0:
            if count<dep_count:
                new_y.append(y[i])
                new_X.append(X[i])
                count+=1
        else:
            new_y.append(y[i])
            new_X.append(X[i])
    return np.array(new_X), np.array(new_y)

In [ ]:
X, y =balance_data(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=35)

In [ ]:
np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)

In [3]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [4]:
def eval_confusion_matrix(actual, predicted):
    TN = 0
    FP = 0
    FN = 0
    TP = 0  
    for i in range(len(actual)):
        if actual[i]==0 or actual[i]==1:
            if actual[i]==predicted[i]:
                if actual[i]==1:
                    TP+=1
                elif actual[i]==0:
                    TN+=1
            else:
                if predicted[i]==1:
                    FP+=1
                elif predicted[i]==0:
                    FN+=1
    
    return TN, FP, FN, TP

def eval_recall(TP, FN):
    recall = TP/(TP+FN)
    return recall

def eval_acc(TN, FP, FN, TP):
    acc = (TN+TP)/(TN+FP+FN+TP)
    return acc

def eval_precision(TP, FP):
    if(TP+FP==0):
        return 0
    prec = TP/(TP+FP)
    return prec

def eval_f1(precision, recall):
    if(precision+recall==0):
        return 0
    f1 = 2*(precision*recall)/(precision+recall)
    return f1

def evaluate(actual, predicted):
    TN, FP, FN, TP = eval_confusion_matrix(actual, predicted)
    recall = eval_recall(TP, FN)
    prec = eval_precision(TP, FP)
    f1 = eval_f1(prec, recall)
    acc= eval_acc(TN, FP, FN, TP)
    print("Recall: ", recall)
    print("Precision: ", prec)
    print("F1 Score: ",f1)
    print("Accuracy: ", acc)
    return f1

### Testing with Logistic Regression

In [46]:
reg = linear_model.LogisticRegression()

In [47]:
reg.fit(X_train, y_train)

C:\Users\65842\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [48]:
probas = reg.predict_proba(X_test)
print(probas)

[[0.00716101 0.99283899]
 [0.86879051 0.13120949]
 [0.1858073  0.8141927 ]
 [0.54713603 0.45286397]
 [0.25216961 0.74783039]
 [0.77146172 0.22853828]
 [0.9620358  0.0379642 ]
 [0.02938359 0.97061641]
 [0.16067899 0.83932101]
 [0.99122646 0.00877354]
 [0.03993019 0.96006981]]


In [49]:
y_pred = reg.predict(X_test)

In [50]:
y_pred

array([1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

In [51]:
y_test

array([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

In [52]:
evaluate(y_test, y_pred)

Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454


0.5454545454545454

### Testing with Decision Tree

In [53]:
from sklearn.tree import DecisionTreeClassifier

In [54]:
def find_tree(upper_depth, upper_leaf):
    best_depth = 0
    best_leaf = 0
    best_f1 = 0
    for i in range(1, upper_depth+1):
        for j in range(1, upper_leaf+1):
            model = DecisionTreeClassifier(max_depth=i, min_samples_leaf=j) 
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            print("Max_depth: {} Min_samples_leaf: {}".format(i, j))
            f1 = evaluate(y_test, y_pred)
            if f1>best_f1:
                best_f1 = f1
                best_depth = i
                best_leaf = j
    return best_depth, best_leaf, best_f1

In [55]:
best_depth, best_leaf, best_f1 = find_tree(100, 50)

Max_depth: 1 Min_samples_leaf: 1
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 2
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 3
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 4
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 5
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 6
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 7
Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364
Max_depth: 1 Min_samples_leaf: 8
Recall: 

Max_depth: 3 Min_samples_leaf: 4
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 3 Min_samples_leaf: 5
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 3 Min_samples_leaf: 6
Recall:  0.4
Precision:  0.3333333333333333
F1 Score:  0.3636363636363636
Accuracy:  0.36363636363636365
Max_depth: 3 Min_samples_leaf: 7
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 3 Min_samples_leaf: 8
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 3 Min_samples_leaf: 9
Recall:  0.4
Precision:  0.3333333333333333
F1 Score:  0.3636363636363636
Accuracy:  0.36363636363636365
Max_depth: 3 Min_samples_leaf: 10
Recall:  0.4
Precision:  0.3333333333333333
F1 Score:  0.3636363636363636
Accuracy:  0.36363636363636365
Max_depth: 3 Min_samples_leaf: 11
Rec

Max_depth: 4 Min_samples_leaf: 45
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 4 Min_samples_leaf: 46
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 4 Min_samples_leaf: 47
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 4 Min_samples_leaf: 48
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 4 Min_samples_leaf: 49
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 4 Min_samples_leaf: 50
Recall:  1.0
Precision:  0.45454545454545453
F1 Score:  0.625
Accuracy:  0.45454545454545453
Max_depth: 5 Min_samples_leaf: 1
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 5 Min_samples_leaf: 2
Recall:  0.6
Precision:  0.42857142857142855
F1 Score:  0.5
Accuracy:  0.45454545454545453
Max_dept

Max_depth: 6 Min_samples_leaf: 43
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 6 Min_samples_leaf: 44
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 6 Min_samples_leaf: 45
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 6 Min_samples_leaf: 46
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 6 Min_samples_leaf: 47
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 6 Min_samples_leaf: 48
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 6 Min_samples_leaf: 49
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 6 Min_samples_leaf: 50
Recall:  1.0
Precision:  0.45454545454545453
F1 Score:  0.625
Accuracy:  0.45454545454545453
Max_dept

Max_depth: 8 Min_samples_leaf: 30
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 8 Min_samples_leaf: 31
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 8 Min_samples_leaf: 32
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 8 Min_samples_leaf: 33
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 8 Min_samples_leaf: 34
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 8 Min_samples_leaf: 35
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 8 Min_samples_leaf: 36
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 8 Min_samples_leaf: 37
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_

...

Max_depth: 93 Min_samples_leaf: 2
Recall:  0.6
Precision:  0.42857142857142855
F1 Score:  0.5
Accuracy:  0.45454545454545453
Max_depth: 93 Min_samples_leaf: 3
Recall:  0.4
Precision:  0.3333333333333333
F1 Score:  0.3636363636363636
Accuracy:  0.36363636363636365
Max_depth: 93 Min_samples_leaf: 4
Recall:  0.8
Precision:  0.4444444444444444
F1 Score:  0.5714285714285714
Accuracy:  0.45454545454545453
Max_depth: 93 Min_samples_leaf: 5
Recall:  0.8
Precision:  0.4444444444444444
F1 Score:  0.5714285714285714
Accuracy:  0.45454545454545453
Max_depth: 93 Min_samples_leaf: 6
Recall:  0.8
Precision:  0.4444444444444444
F1 Score:  0.5714285714285714
Accuracy:  0.45454545454545453
Max_depth: 93 Min_samples_leaf: 7
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 93 Min_samples_leaf: 8
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 93 Min_samples_leaf: 9
Recall:  

Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 94 Min_samples_leaf: 46
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 94 Min_samples_leaf: 47
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 94 Min_samples_leaf: 48
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 94 Min_samples_leaf: 49
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 94 Min_samples_leaf: 50
Recall:  1.0
Precision:  0.45454545454545453
F1 Score:  0.625
Accuracy:  0.45454545454545453
Max_depth: 95 Min_samples_leaf: 1
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_depth: 95 Min_samples_leaf: 2
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Max_dep

Max_depth: 96 Min_samples_leaf: 42
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 43
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 44
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 45
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 46
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 96 Min_samples_leaf: 47
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 96 Min_samples_leaf: 48
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 96 Min_samples_leaf: 49
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_

Accuracy:  0.5454545454545454
Max_depth: 98 Min_samples_leaf: 32
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 98 Min_samples_leaf: 33
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 98 Min_samples_leaf: 34
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 98 Min_samples_leaf: 35
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 98 Min_samples_leaf: 36
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 98 Min_samples_leaf: 37
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 98 Min_samples_leaf: 38
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 98 Min_samples_leaf: 39
Recall:  0.2
Precision:  0.2
F1 Score:  0.2000000000

Max_depth: 100 Min_samples_leaf: 33
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 100 Min_samples_leaf: 34
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 100 Min_samples_leaf: 35
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 100 Min_samples_leaf: 36
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 100 Min_samples_leaf: 37
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 100 Min_samples_leaf: 38
Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365
Max_depth: 100 Min_samples_leaf: 39
Recall:  0.2
Precision:  0.2
F1 Score:  0.20000000000000004
Accuracy:  0.2727272727272727
Max_depth: 100 Min_samples_leaf: 40
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5

In [56]:
print(best_depth, best_leaf)

1 1


In [57]:
model = DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf=best_leaf) 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = evaluate(y_test, y_pred)

Recall:  1.0
Precision:  0.5555555555555556
F1 Score:  0.7142857142857143
Accuracy:  0.6363636363636364


In [58]:
print(y_pred)
print(y_test)

[1 0 1 1 1 1 0 1 1 1 1]
[0 0 1 1 1 0 0 0 1 1 0]


### Testing with kNN

In [59]:
from sklearn.neighbors import KNeighborsClassifier

In [60]:
def find_k(upper_k):
    best_k = 0
    best_f1 = 0
    for k in range(1,upper_k+1):
        neigh = KNeighborsClassifier(n_neighbors=k)
        neigh.fit(X_train, y_train)
        y_pred = neigh.predict(X_test)
        print("Value of k: ",k)
        f1 = evaluate(y_test, y_pred)
        if f1>best_f1:
            best_k = k
            best_f1 = f1
    return best_k, best_f1

In [61]:
best_k, best_f1=find_k(10)

Value of k:  1
Recall:  0.4
Precision:  0.2857142857142857
F1 Score:  0.3333333333333333
Accuracy:  0.2727272727272727
Value of k:  2
Recall:  0.0
Precision:  0.0
F1 Score:  0
Accuracy:  0.45454545454545453
Value of k:  3
Recall:  0.2
Precision:  0.16666666666666666
F1 Score:  0.1818181818181818
Accuracy:  0.18181818181818182
Value of k:  4
Recall:  0.0
Precision:  0.0
F1 Score:  0
Accuracy:  0.2727272727272727
Value of k:  5
Recall:  0.0
Precision:  0.0
F1 Score:  0
Accuracy:  0.18181818181818182
Value of k:  6
Recall:  0.0
Precision:  0.0
F1 Score:  0
Accuracy:  0.36363636363636365
Value of k:  7
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Value of k:  8
Recall:  0.0
Precision:  0.0
F1 Score:  0
Accuracy:  0.36363636363636365
Value of k:  9
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Value of k:  10
Recall:  0.2
Precision:  0.3333333333333333
F1 Score:  0.25
Accuracy:  0.45454545454545453


In [62]:
neigh = KNeighborsClassifier(n_neighbors=best_k)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454


0.4444444444444445

In [63]:
print(y_pred)
print(y_test)

[1 0 1 1 0 0 0 1 0 0 0]
[0 0 1 1 1 0 0 0 1 1 0]


### Testing with gaussian naive bayes

In [64]:
from sklearn.naive_bayes import GaussianNB

In [65]:
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.2
Precision:  0.25
F1 Score:  0.22222222222222224
Accuracy:  0.36363636363636365


0.22222222222222224

In [66]:
print(y_pred)
print(y_test)

[1 0 1 0 0 0 0 1 0 0 1]
[0 0 1 1 1 0 0 0 1 1 0]


### Testing with SVM

In [67]:
from sklearn.svm import SVC

In [68]:
model = SVC(C=10, gamma='auto', kernel='rbf')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.8
Precision:  0.4
F1 Score:  0.5333333333333333
Accuracy:  0.36363636363636365


0.5333333333333333

### Testing with AdaBoost

In [69]:
from sklearn.ensemble import AdaBoostClassifier

In [70]:
model = AdaBoostClassifier(n_estimators=200, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454


0.4444444444444445

### Testing with Random forest

In [71]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
model = RandomForestClassifier(n_estimators = 40)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.4
Precision:  0.3333333333333333
F1 Score:  0.3636363636363636
Accuracy:  0.36363636363636365


0.3636363636363636

In [73]:
print(y_pred)
print(y_test)

[1 0 1 0 1 1 0 1 0 0 1]
[0 0 1 1 1 0 0 0 1 1 0]


In [74]:
model.predict_proba(X_test)

array([[0.475, 0.525],
       [0.65 , 0.35 ],
       [0.45 , 0.55 ],
       [0.625, 0.375],
       [0.425, 0.575],
       [0.475, 0.525],
       [0.85 , 0.15 ],
       [0.3  , 0.7  ],
       [0.6  , 0.4  ],
       [0.675, 0.325],
       [0.325, 0.675]])

### Testing with Neural network

In [5]:
y_train = to_categorical(y_train)

In [13]:
model = Sequential()
model.add(Dense(5, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [14]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [15]:
checkpoint_filepath = 'model1.h5'
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 10)
checkpoint =  ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [16]:
history = model.fit(X_train, y_train, epochs=50, batch_size=1, validation_split=0.1, callbacks=[es, checkpoint])

Train on 89 samples, validate on 10 samples
Epoch 1/50
89/89 [==============================] - 1s 6ms/step - loss: 4.0213 - accuracy: 0.4607 - val_loss: 2.2516 - val_accuracy: 0.7000
Epoch 2/50
89/89 [==============================] - 0s 5ms/step - loss: 3.5480 - accuracy: 0.4045 - val_loss: 2.0726 - val_accuracy: 0.5000
Epoch 3/50
89/89 [==============================] - 0s 4ms/step - loss: 3.3219 - accuracy: 0.4494 - val_loss: 1.7341 - val_accuracy: 0.7000
Epoch 4/50
89/89 [==============================] - 0s 4ms/step - loss: 3.0313 - accuracy: 0.3933 - val_loss: 2.0410 - val_accuracy: 0.6000
Epoch 5/50
89/89 [==============================] - 0s 4ms/step - loss: 2.5773 - accuracy: 0.4157 - val_loss: 1.2118 - val_accuracy: 0.7000
Epoch 6/50
89/89 [==============================] - 0s 4ms/step - loss: 2.4416 - accuracy: 0.4494 - val_loss: 1.1785 - val_accuracy: 0.6000
Epoch 7/50
89/89 [==============================] - 0s 4ms/step - loss: 1.9584 - accuracy: 0.4494 - val_loss: 1.3922

In [17]:
model = load_model(checkpoint_filepath)

In [18]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred)
print(y_test)

[0 1 0 1 0 0 1 1 1 0 1]
[0 0 1 1 1 0 0 0 1 1 0]


In [19]:
evaluate(y_test, y_pred)

Recall:  0.4
Precision:  0.3333333333333333
F1 Score:  0.3636363636363636
Accuracy:  0.36363636363636365


0.3636363636363636